# Model

In [1]:
import tensorflow as tf

In [2]:
D = 16
tf.reset_default_graph()

In [3]:
with tf.name_scope("constant"):
    t = tf.placeholder(tf.float64, shape=())
    H = tf.reshape(tf.constant([[0.25,0,0,0],[0,-0.25,0.5,0],[0,0.5,-0.25,0],[0,0,0,0.25]],
                               dtype=tf.float64),[2,2,2,2],name="Hamiltonian")
    I = tf.reshape(tf.constant([[1.,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]],
                               dtype=tf.float64),[2,2,2,2],name="Identity")
    expH = I + t * H

In [4]:
with tf.name_scope("variables"):
    A = tf.placeholder(shape=[D, D, 2], dtype=tf.float64,name="A")
    B = tf.placeholder(shape=[D, D, 2], dtype=tf.float64, name="B")
    EAB = tf.placeholder(shape=[D], dtype=tf.float64, name="EAB")
    EBA = tf.placeholder(shape=[D], dtype=tf.float64, name="EBA")
    L = tf.placeholder(shape=[D, D], dtype=tf.float64, name="L")
    R = tf.placeholder(shape=[D, D], dtype=tf.float64, name="R")

In [5]:
with tf.name_scope("parameter"):
    REAB = tf.reciprocal(EAB,name="REAB")
    REBA = tf.reciprocal(EBA,name="REBA")
    
    EA = tf.multiply(tf.multiply(A,tf.reshape(EBA,[D,1,1])),tf.reshape(EAB,[1,D,1]),name="EA")
    EB = tf.multiply(tf.multiply(B,tf.reshape(EAB,[D,1,1])),tf.reshape(EBA,[1,D,1]),name="EB")
    
    AB = tf.tensordot(EA,EB,[[1],[0]],name="AB")
    BA = tf.tensordot(EB,EA,[[1],[0]],name="BA")

In [6]:
def d_max(t):
    with tf.name_scope("d_max"):
        return t/tf.reduce_max(tf.abs(t))

In [7]:
with tf.name_scope("updateAB"):
    HAB = tf.tensordot(AB,expH,[[1,3],[0,1]],name="HAB")
    S, U, V = tf.svd(tf.reshape(tf.transpose(HAB,[0,2,1,3]),[2*D,2*D]))
    ABnEAB = tf.sqrt(S[:D],name="nE")
    ABnA = tf.transpose(tf.multiply(tf.reshape(U[:,:D],[D,2,D]),tf.reshape(REBA,[D,1,1])),[0,2,1],name="nA")
    ABnB = tf.transpose(tf.multiply(tf.reshape(V[:,:D],[D,2,D]),tf.reshape(REBA,[D,1,1])),[2,0,1],name="nB")
    updateAB = tf.tuple((d_max(ABnEAB),d_max(ABnA),d_max(ABnB)),name="updateAB")

with tf.name_scope("updateBA"):
    HBA = tf.tensordot(BA,expH,[[1,3],[0,1]],name="HBA")
    S, U, V = tf.svd(tf.reshape(tf.transpose(HBA,[0,2,1,3]),[2*D,2*D]))
    BAnEBA = tf.sqrt(S[:D],name="nE")
    BAnB = tf.transpose(tf.multiply(tf.reshape(U[:,:D],[D,2,D]),tf.reshape(REAB,[D,1,1])),[0,2,1],name="nB")
    BAnA = tf.transpose(tf.multiply(tf.reshape(V[:,:D],[D,2,D]),tf.reshape(REAB,[D,1,1])),[2,0,1],name="nA")
    updateBA = tf.tuple((d_max(BAnEBA),d_max(BAnB),d_max(BAnA)),name="updateBA")

In [8]:
with tf.name_scope("updateL"):
    LA = tf.tensordot(tf.tensordot(L,EA,[[1],[0]]),EA,[[0,2],[0,2]],name="LA")
    LAB = tf.tensordot(tf.tensordot(LA,EB,[[1],[0]]),EB,[[0,2],[0,2]],name="LAB")
    
    updateL = d_max(LAB)
        
with tf.name_scope("updateR"):
    BR = tf.tensordot(tf.tensordot(R,EB,[[1],[1]]),EB,[[0,2],[1,2]],name="BR")
    ABR = tf.tensordot(tf.tensordot(BR,EA,[[1],[1]]),EA,[[0,2],[1,2]],name="ABR")
    updateR = d_max(ABR)

In [9]:
with tf.name_scope("Energy"):
    LA = tf.tensordot(L,EA,[[0],[0]])
    LAB = tf.tensordot(LA,EB,[[1],[0]])
    LABR = tf.tensordot(LAB,R,[[2],[0]])
    LABRA = tf.tensordot(LABR,EA,[[0],[0]])
    LABRAB = tf.tensordot(LABRA,EB,[[3,2],[0,1]])
    E_AB = tf.tensordot(LABRAB,H,[[0,1,2,3],[0,1,2,3]])/tf.tensordot(LABRAB,I,[[0,1,2,3],[0,1,2,3]])

    LABA = tf.tensordot(LAB,EA,[[2],[0]])
    LABAA = tf.tensordot(LABA,EA,[[0,1],[0,2]])
    LABAAB = tf.tensordot(LABAA,EB,[[3],[0]])
    LABAABA = tf.tensordot(LABAAB,EA,[[3],[0]])
    LABAABAB = tf.tensordot(LABAABA,EB,[[4],[0]])
    LABAABABB = tf.tensordot(LABAABAB,EB,[[1,6],[0,2]])
    LABAABABBR = tf.tensordot(LABAABABB,R,[[5,4],[0,1]])
    E_BA = tf.tensordot(LABAABABBR,H,[[0,1,2,3],[0,1,2,3]])/tf.tensordot(LABAABABBR,I,[[0,1,2,3],[0,1,2,3]])

    E = (E_AB+E_BA)/2

In [10]:
tf.add_to_collection("t",t)
tf.add_to_collection("A",A)
tf.add_to_collection("B",B)
tf.add_to_collection("EAB",EAB)
tf.add_to_collection("EBA",EBA)
tf.add_to_collection("L",L)
tf.add_to_collection("R",R)
tf.add_to_collection("updateAB",updateAB[0])
tf.add_to_collection("updateAB",updateAB[1])
tf.add_to_collection("updateAB",updateAB[2])
tf.add_to_collection("updateBA",updateBA[0])
tf.add_to_collection("updateBA",updateBA[1])
tf.add_to_collection("updateBA",updateBA[2])
tf.add_to_collection("updateL",updateL)
tf.add_to_collection("updateR",updateR)
tf.add_to_collection("E",E)

In [11]:
meta_graph_def = tf.train.export_meta_graph(filename='./MPS-%d.pbtxt'%D, as_text=True)
file_writer = tf.summary.FileWriter('.', tf.get_default_graph())

# Running

In [12]:
import numpy as np
import tensorflow as tf

In [13]:
class MPS():
    def __init__(self,D):
        self.sess = tf.get_default_session()
        self.g = tf.get_default_graph()
        tf.train.import_meta_graph("./MPS-%d.pbtxt"%D)
        self.dA = np.random.random([D,D,2])
        self.dB = np.random.random([D,D,2])
        self.dEAB = np.ones([D])
        self.dEBA = np.ones([D])
        self.dL = np.random.random([D,D])
        self.dR = np.random.random([D,D])
        self.ep = 0.1
        self.t = self.g.get_collection("t")[-1]
        self.A = self.g.get_collection("A")[-1]
        self.B = self.g.get_collection("B")[-1]
        self.EAB = self.g.get_collection("EAB")[-1]
        self.EBA = self.g.get_collection("EBA")[-1]
        self.L = self.g.get_collection("L")[-1]
        self.R = self.g.get_collection("R")[-1]
        self.updateAB = self.g.get_collection("updateAB")[-3:]
        self.updateBA = self.g.get_collection("updateBA")[-3:]
        self.updateL = self.g.get_collection("updateL")[-1]
        self.updateR = self.g.get_collection("updateR")[-1]
        self.E = self.g.get_collection("E")[-1]

    def run_updateAB(self):
        self.dEAB, self.dA, self.dB = self.sess.run(self.updateAB,feed_dict={
            self.t:-4*self.ep, self.A:self.dA, self.B:self.dB, self.EAB:self.dEAB, self.EBA:self.dEBA})
    def run_updateBA(self):
        self.dEBA, self.dB, self.dA = self.sess.run(self.updateBA,feed_dict={
            self.t:-4*self.ep, self.A:self.dA, self.B:self.dB, self.EAB:self.dEAB, self.EBA:self.dEBA})
    def run_updateL(self):
        self.dL = self.sess.run(self.updateL,feed_dict={
            self.A:self.dA, self.B:self.dB, self.EAB:self.dEAB, self.EBA:self.dEBA, self.L:self.dL})
    def run_updateR(self):
        self.dR = self.sess.run(self.updateR,feed_dict={
            self.A:self.dA, self.B:self.dB, self.EAB:self.dEAB, self.EBA:self.dEBA, self.R:self.dR})
    def get_energy(self):
        return self.sess.run(self.E,feed_dict={
            self.A:self.dA, self.B:self.dB, self.EAB:self.dEAB, self.EBA:self.dEBA, self.L:self.dL, self.R:self.dR})

In [14]:
with tf.Graph().as_default(), tf.Session().as_default():
    mps = MPS(D)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [15]:
%%time
for _ in xrange(10):
    for _ in xrange(100):
        mps.run_updateAB()
        mps.run_updateBA()
        mps.run_updateBA()
        mps.run_updateAB()
    tmp = 0
    #while abs(mps.get_energy()-tmp)>0.01:
    for _ in [0]:
        for _ in xrange(100):
            mps.run_updateL()
            mps.run_updateR()
        tmp = mps.get_energy()
    print tmp

-0.435446630909
-0.435959017232
-0.43586721527
-0.435862814711
-0.435862188572
-0.435861199471
-0.435861609592
-0.435861508722
-0.435861442868
-0.435861432298
CPU times: user 16.1 s, sys: 1.32 s, total: 17.4 s
Wall time: 9.7 s
